NLP Model on New Data Rev 1.0

This program attempts to load a nlp classifier model and use it on a new/different data set other than that which created the model.

The export/import of the model and the vocabulary file are done through the pickle library
The extension of the model file and the vocabulary file is .sav

The first step is to load the data to be analyzed by the model. (.csv file)

The second step is to load the actual model and the vocabulary file. (.sav files) Both files are necessary.

Each are created when the model is created and saved through the use of pickle, which in essence saves the file as a binary file. Both of these files must be saved when the model is created.

https://medium.com/@maziarizadi/pickle-your-model-in-python-2bbe7dba2bbb
https://www.kdnuggets.com/2019/11/create-vocabulary-nlp-tasks-python.html

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk.classify
from nltk import NaiveBayesClassifier
analyzer = SentimentIntensityAnalyzer()
import time
import pickle

###########################################################
#### METHODS                                           ####
###########################################################

# displays a list of file with on a csv suffix       
def list_dir_files(relevant_path):
    # https://clay-atlas.com/us/blog/2019/10/27/python-english-tutorial-solved-unicodeescape-error-escape-syntaxerror/?doing_wp_cron=1618286551.1528689861297607421875
    #need to change \ to /

    import os
    
    included_extensions = ['csv']
    file_names = [fn for fn in os.listdir(relevant_path) # uses os.listdir to display only .csv files
              if any(fn.endswith(ext) for ext in included_extensions)]

    print('Path: ', relevant_path)

    for f in file_names:
        print(f)
        
# initializes the dataframe "df" and imports the csv into df; 
# the argument is the name/address of the file.
# https://stackoverflow.com/questions/33440805/pandas-dataframe-read-csv-on-bad-data
def getData(name):
    df1 = pd.DataFrame() # defines df1 as a dataframe
    df1 = pd.read_csv(name, header = 0)
    return df1


############################################################
####   MAIN                                             ####
############################################################

#Loads a specific model and vocabulary file:
#ModelFileToLoad = 'C:/Users/pstri/OneDrive/Documents/Personal/Kokoro/NLTK/Code Project/NLP Models/preprocessed r_a tech stockTwit 03112021Model.sav'
#VocabFileToLoad = 'C:/Users/pstri/OneDrive/Documents/Personal/Kokoro/NLTK/Code Project/NLP Models/preprocessed r_a tech stockTwit 03112021Vocab.sav'

ModelFileToLoad = 'preprocessed r_a tech stockTwit 03112021Model.sav'
VocabFileToLoad = 'preprocessed r_a tech stockTwit 03112021Vocab.sav'

loaded_model = pickle.load(open(ModelFileToLoad, 'rb'))
loaded_vocabulary = pickle.load(open(VocabFileToLoad, 'rb'))

'''#Calls getData to import the csv into the dataframe. This file is has the data that we want to perform the sentiment analysis on.
relevant_path = 'C:/Users/pstri/OneDrive/Documents/Personal/Kokoro/NLTK/Code Project/Scraped Files'

print('Here are the list of csv files to choose from: ')
list_dir_files(relevant_path) # method to list files in the directory where this file is found.
'''
time.sleep(1)

# name = input('\nWhat is the name of the csv file you want to import? \n')

name = 'preprocessed r_d r_a r_e_o tech stockTwit 03112021.csv'
dfAPI = getData(name)

fileName = name

####
# Need to convert sanitized data into vectors
####

# converts the df columns of body and the label (compound or sentiment_number) into one list for each column
#this is needed to be able to create the 
tech_twits_text = dfAPI['body'].tolist() #'body' is the column that contains the training data

''' There is no need for the following steps because they are captured in the 'loaded_vocabulary' file.
The vocabulary was created when the model was created.

from sklearn.feature_extraction.text import CountVectorizer
counter = CountVectorizer() #creates a CountVectorizer named counter
counter.fit(tech_twits_text) #teaches the counter the vocabulary

The loaded_vocabulary' file is set equal to counter and then counter is used to transform the data into a Count Vector.
This is just like when the model was created. The model then is applied to the Count Vector to determine sentiment.

'''
counter = loaded_vocabulary

tech_twits_text_counts = counter.transform(tech_twits_text) #transforms train_data to a Count Vector which will be the input to the model for sentiment analysis

predictions = loaded_model.predict(tech_twits_text_counts) #predicts the value (label/sentiment) from the tech_twits_text_counts 

print('predictions = ', predictions)


predictions =  [1. 1. 1. ... 1. 0. 0.]


In [3]:
len(predictions) #gives the length of the array

type(predictions) #gives the data type of predictions

copy_predictions = predictions #makes a copy of predictions

df = pd.DataFrame(data = copy_predictions, columns = ['Sentiment']) #creates a dataframe from 'copy_predictions'

display(df)

#itemizes the predicted sentiment values
bullish = 0
neutral = 0
bearish = 0

i = 0

while i < len(df):
    if df.iloc[i,0] == 1:
        bullish += 1
    elif df.iloc[i,0] == 0:
        neutral += 1
    elif df.iloc[i,0] == -1:
        bearish = 0
    
    i += 1
    
print('bullish = ', bullish)
print('neutral = ', neutral)
print('bearish = ', bearish)
    

,Sentiment
0,1.0
1,1.0
2,1.0
3,1.0
4,0.0
...,...
1286,0.0
1287,1.0
1288,1.0
1289,0.0


bullish =  663
neutral =  580
bearish =  0


In [ ]:
'''def getData(csv_name):
    df1 = pd.DataFrame() # defines df1 as a dataframe
    df1 = pd.read_csv(csv_name, header = 0, encoding = 'utf-8') # reads the csv file into df1 (dataframe)
    return df1

def df_to_model_accuracy(df, filename):
    # 160 converts the df columns of body and the label (compound or sentiment_number) into one list for each column
    #this is needed to be able to create the 
    tech_twits_text = df['body'].tolist() # 'body' is the column that contains the training data
    compound_bin_labels = df['compound_bin'].tolist() # 'compound_bin are the numbers that represent the nltk Vader compound sentiment numbers as 1,0,-1'
    sentiment_number_labels = df['sentiment_number'].tolist() #'sentiment_number' column is the integer tranlsation from the stocktwits 'bullish', 'None', 'bearish' sentiment rankings
    #labels = df['modified_rating'].tolist()
    #labels = df['Rating'].tolist()
    #labels = df['raw_compound'].tolist()

    # 170 divides the whole data set into a 80/20 split into a training set and a test set
    from sklearn.model_selection import train_test_split
    # train_labels are from Vader's compound bin
    train_data, test_data, train_labels, test_labels = train_test_split(tech_twits_text, compound_bin_labels, test_size = 0.2, random_state = 1)
    
    # sntrain_labels are from the stocktwits sentiment self reported sentiment values.
    sntrain_data, sntest_data, sntrain_labels, sntest_labels = train_test_split(tech_twits_text, sentiment_number_labels, test_size = 0.2, random_state = 1)

    # 190 Converts the text into vectors based on word usage
    from sklearn.feature_extraction.text import CountVectorizer
    counter = CountVectorizer() #creates a CountVectorizer named counter
    counter.fit(train_data) #teaches the counter our vocabulary
    
    ## for VADER
    train_counts = counter.transform(train_data) #transforms train_data to a Count Vector
    test_counts = counter.transform(test_data) #transforms test_data to a Count Vector

    ## for Stocktwits sentiment
    sntrain_counts = counter.transform(sntrain_data) #transforms train_data to a Count Vector
    sntest_counts = counter.transform(sntest_data) #transforms test_data to a Count Vector    
    
    # 200 creates the model based on the vectors and applies (predictions) the model to the test data set
    #https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
    from sklearn.naive_bayes import MultinomialNB
    classifierVader = MultinomialNB()
    classifierStocktwits = MultinomialNB()
    
    ## for VADER
    classifierVader.fit(train_counts, train_labels)
    predictions = classifierVader.predict(test_counts) # predicts the value (label/sentiment) from the test_counts is the test_data 

    ## for Stocktwits sentiment
    classifierStocktwits.fit(sntrain_counts, sntrain_labels)
    snpredictions = classifierStocktwits.predict(test_counts) # predicts the value (label/sentiment) from the test_counts is the test_data 
    
    print('This compares the multinomial naive bayes sentiment model [MNBSM] using different inputs and outputs')
    print('The inputs were either produced by Vader or taken directly from Stocktwits.')
    print('The labels were either produced by Vader or taken directily from Stocktwits.')
    print('All comparisons are with the MNBSM but with different inputs and different labels.')
    
    # 230 compares the accuracy of the predicted values and the real test values (labels)
    from sklearn.metrics import accuracy_score
    print('accuracy score of Stocktwits model to Stocktwits: \n', accuracy_score(sntest_labels, snpredictions), '\n')  
    print('accuracy score of Vader to Vader: \n', accuracy_score(test_labels, predictions), '\n')
    print('accuracy score of Vader to Stocktwits: \n', accuracy_score(sntest_labels, predictions), '\n')

    # 240 compares the precision of the predicted values to the real test values (labels)
    #https://nlpforhackers.io/classification-performance-metrics/
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html
    from sklearn.metrics import precision_score
    print('precision score of Stocktwits model to Stocktwits: \n', precision_score(sntest_labels, snpredictions, average = 'micro'), '\n')  
    print('precision score of Vader to Vader: \n', precision_score(test_labels, predictions, average = 'micro'), '\n')
    print('precision score of Vader to Stocktwits: \n', precision_score(sntest_labels, predictions, average = 'micro'), '\n')
    
    # 250 Confusion Matrix
    from sklearn.metrics import confusion_matrix
    print('confusion matrix of Stocktwits model to Stocktwits: \n', confusion_matrix(sntest_labels, snpredictions), '\n')
    print('confusion matrix of Vader to Vader: \n', confusion_matrix(test_labels, predictions), '\n')
    print('confusion matrix of Vader to Stocktwits: \n', confusion_matrix(sntest_labels, predictions), '\n')
    
    #260 Classification Report
    from sklearn.metrics import classification_report
    print('classification report of Stocktwits to Stocktwits: \n', classification_report(sntest_labels, snpredictions))
    print('classification report of Vader to Vader: \n', classification_report(test_labels, predictions))
    print('classification report of Vader to Stocktwits: \n', classification_report(sntest_labels, predictions))

    
    ########
    # EXPORT MODEL
    ########

    csv_filename = filename

    model_filename = csv_filename.replace('.csv', '.sav')

    print('The filename of the model is: ', model_filename)

    pickle.dump(classifierStocktwits, open(model_filename, 'wb'))

    
    # some time later...
 
    # load the model from disk
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(sntrain_counts, sntrain_labels)
    print(result)
    
    
    
# prints out the names of the files in the directory
def list_dir_files(relevant_path):
    #https://clay-atlas.com/us/blog/2019/10/27/python-english-tutorial-solved-unicodeescape-error-escape-syntaxerror/?doing_wp_cron=1618286551.1528689861297607421875
    #need to change \ to /

    import os

    included_extensions = ['csv']
    file_names = [fn for fn in os.listdir(relevant_path)
              if any(fn.endswith(ext) for ext in included_extensions)]

    for f in file_names:
        print(f)'''

https://stackoverflow.com/questions/61967427/i-am-getting-an-vocabulary-not-fitted-or-provided
https://www.programcreek.com/python/example/103577/sklearn.exceptions.NotFittedError

https://stackoverflow.com/questions/60472925/python-scikit-svm-vocabulary-not-fitted-or-provided
you need to use the vectorizer on which you have trained the model. in your code you are creating object for tfidfvectorizer and than using it with transform, which will throw the error.. so you need to save vectorizer when training the model and than using the same vectorizer for prediction – qaiser Mar 2 '20 at 7:01 

In [4]:
!pip list

Package                            Version             Location
---------------------------------- ------------------- --------------------------------------------------------------------------
-                                  ipy
-cipy                              1.5.0
-qdm                               4.47.0
aiodns                             2.0.0
aiohttp                            3.7.3
aiohttp-socks                      0.5.5
alabaster                          0.7.12
alembic                            1.4.1
anaconda-client                    1.7.2
anaconda-navigator                 1.9.12
anaconda-project                   0.8.3
appmode                            0.8.0
argh                               0.26.2
as                                 0.1
asn1crypto                         1.3.0
astroid                            2.4.2
astropy                            4.0.1.post1
async-timeout                      3.0.1
atomicwrites                       1.4.0
attrs                

In [ ]:
import pandas as pd  
pandas                             1.2.4

import numpy as np
numpy                              1.19.5

import nltk
nltk                               3.5

from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk.classify
from nltk import NaiveBayesClassifier
analyzer = SentimentIntensityAnalyzer()
import time

import pickle
pickleshare                        0.7.5

In [13]:
import pandas
print(pandas.__version__)

1.2.4


In [8]:
import numpy
print(numpy.__version__)

1.19.5


In [14]:
import pickle
print(pickleshare.__version__)

NameError: name 'pickleshare' is not defined